In [13]:
# === [0. Imports] ===
import os
import sys
import pandas as pd
from utils.experiment_tracker import ExperimentTracker
from utils.constants import METRICS_DB_PATH, CLEANED_DIR
from utils.model_evaluator import ModelEvaluator
from utils.model_table import ModelComparativeTable

# === [1. Dynamically build absolute path to pre-study CSV file] ===
notebook_dir = os.getcwd()
csv_relative_path = os.path.join("data", "ml_pre_study_metrics", "model_metrics.csv")
model_pre_study_path = os.path.abspath(os.path.join(notebook_dir, "..", "..", csv_relative_path))

# === [2. Check that the CSV file exists before loading it] ===
if not os.path.isfile(model_pre_study_path):
    raise FileNotFoundError(f"❌ File not found: {model_pre_study_path}")
print(f"✅ Found file: {model_pre_study_path}")

# === [3. Try to display pre-study summary from CSV file] ===
mcp = None
try:
    mcp = ModelComparativeTable()
    
    # Vérifie si le fichier contient des données exploitables
    df_csv = pd.read_csv(model_pre_study_path)
    if df_csv.empty or "r2" not in df_csv.columns or df_csv["r2"].dropna().empty:
        raise ValueError("CSV file is empty or missing valid 'r2' values.")

    mcp.display_model_summary_pre_study(model_pre_study_path)

except Exception as e:
    print("⚠️ Failed to display pre-study summary:", e)

# === [4. Display live summary from ExperimentTracker if available] ===
try:
    if mcp is not None and not mcp.df_all_evals.empty:
        mcp.display_model_summary()
    else:
        print("⚠️ No experiment logs found in SQLite tracker.")
except Exception as e:
    print("⚠️ Failed to display model summary:", e)


✅ Found file: e:\_SoftEng\_BeCode\real-estate-price-predictor\data\ml_pre_study_metrics\model_metrics.csv
⚠️ Failed to display pre-study summary: attempt to get argmax of an empty sequence
⚠️ No experiment logs found in SQLite tracker.
